In [1]:
import csv
import copy
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
market_csv_file = "data/S&P500.csv"
stock_csv_file = "data/stock_clean.csv"
df_market = pd.read_csv(market_csv_file)
df_stock = pd.read_csv(stock_csv_file)

In [7]:
df_stock = df_stock.rename(columns = {df_stock.columns[0]:"date"})
result = df_stock["date"].isin(["19720731"])
t = df_stock["date"][result].index.values[0]
print(t)

119


In [9]:
#Drop columns that have less than pre 120 + post 12 = 132 valid return
tmp_df_stock_all = df_stock[t-119:t+13].dropna(thresh=df_stock[t-119:t+13].shape[0],how='all',axis=1)
#print(tmp_df_stock_all)
#Get the prev
tmp_df_stock = tmp_df_stock_all[0:120]
tmp_df_market = df_market[t-119:t+1]
N = len(tmp_df_stock.columns[1:])

In [12]:
#sample covariance matrix estimator
sample_cov = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).cov()

In [20]:
#market model estimator
y = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).values
y_bar = y.mean()
x = tmp_df_market[tmp_df_market.columns[1:]].astype(np.float64).values
x_bar = x.mean()
beta_1 = np.sum((y-y_bar)*(x),axis=0)/np.sum((x-x_bar)*x,axis=0)
beta_0 = y_bar - beta_1*x_bar
residual = y - (beta_1*x+beta_0)
delta = np.sum(residual**2,axis=0)/(N-2)
D = np.diag(delta)
s_2 = x.var()
F = s_2*beta_1[:,np.newaxis]@beta_1[:,np.newaxis].T + D
#print(F)

In [62]:
#backtest
monthly_returns = []
#range(t,384,12)
for i in tqdm(range(t,t+1,12)):

    #Drop columns that have less than pre 120 + post 12 = 132 valid return
    tmp_df_stock_all = df_stock[i-119:i+13].dropna(thresh=df_stock[i-119:i+13].shape[0],how='all',axis=1)

    #In sample
    tmp_df_stock = tmp_df_stock_all[0:120]
    N = len(tmp_df_stock.columns[1:])
    tmp_df_market = df_market[t-119:t+1]
    
    #sample covariance matrix estimator
    sample_cov = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).cov()
    
    #market model estimator
    y = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).values
    y_bar = y.mean(axis=0)
    x = tmp_df_market[tmp_df_market.columns[1:]].astype(np.float64).values
    x_bar = x.mean()
    beta_1 = np.sum((y-y_bar)*(x-x_bar),axis=0)/np.sum((x-x_bar)*(x-x_bar),axis=0)
    beta_0 = y_bar - beta_1*x_bar
    residual = y - (beta_1*x+beta_0)
    delta = np.sum(residual**2,axis=0)/(N-2)
    D = np.diag(delta)
    s_2 = x.var()
    F = s_2*beta_1[:,np.newaxis]@beta_1[:,np.newaxis].T + D
    
    cov_matrix = F
    #cov_matrix_inv = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    cov_matrix_inv = np.linalg.pinv(cov_matrix)
    #print(cov_matrix_inv)
    
    w = cov_matrix_inv@np.ones((N,1))/(np.ones((1,N))@cov_matrix_inv@np.ones((N,1)))
    mu = tmp_df_stock[tmp_df_stock.columns[1:]].mean()
    #print(mu)
    
    for j in range(12):
        tmp_df_post = tmp_df_stock_all[120+j:121+j]
        tmp_df_post = tmp_df_post[tmp_df_post.columns[1:]]
        #print(tmp_df_post)
        monthly_return = (tmp_df_post.astype(np.float64)+1).prod(axis=0)-1
        monthly_returns.append(monthly_return@w)
    

100%|██████████| 23/23 [00:06<00:00,  3.57it/s]


In [63]:
monthly_returns = np.array(monthly_returns)
monthly_returns.std()*np.sqrt(12)

0.1432967768232372

In [80]:
pi = 0
rou = 0
gamma = 0
T = 120

for i in range(N):
    for j in range(N):
        gamma += (F[i][j]-S[i][j])**2
        
        tmp = 0
        for k in range(T):
            tmp += ((y[k][i]-y_bar[i])*(y[k][j]-y_bar[j])-S[i][j])**2
        pi += tmp/T
        
        if i == j:
            rou += tmp/T
        else:
            stock_i = y[:,[i]]
            stock_j = y[:,[j]]
            s_i0 = cov()
            
            for k in range(T):
                
        
        


SyntaxError: unexpected EOF while parsing (<ipython-input-80-9aa32952226a>, line 23)

In [91]:
stock_i = y[:,[i]]
stock_j = y[:,[j]]

ValueError: object too deep for desired array